<a href="https://colab.research.google.com/github/CharlemagneBrain/Tf_Transform_TFX/blob/main/C2_W2_Lab_2_Feature_Engineering_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Feature Engineering Pipeline


Dans ce laboratoire, vous allez continuer à explorer [Tensorflow Transform](https://www.tensorflow.org/tfx/transform/get_started). Cette fois, ce sera dans le contexte d'un pipeline d'apprentissage machine (ML). Dans les projets de production, vous souhaitez rationaliser les tâches afin de pouvoir améliorer plus facilement votre modèle ou trouver les problèmes qui peuvent survenir. [Tensorflow Extended (TFX)](https://www.tensorflow.org/tfx) fournit des composants qui fonctionnent ensemble pour exécuter les étapes les plus courantes d'un projet d'apprentissage automatique. Si vous souhaitez approfondir les motivations qui sous-tendent TFX et la nécessité des pipelines d'apprentissage automatique, vous pouvez lire [cet article](https://storage.googleapis.com/pub-tools-public-publication-data/pdf/b500d77bc4f518a1165c0ab43c8fac5d2948bc14.pdf) et ce [billet de blog](https://blog.tensorflow.org/2020/09/brief-history-of-tensorflow-extended-tfx.html).

Vous construirez des pipelines de bout en bout dans les prochains cours, mais pour celui-ci, vous n'aborderez que la partie ingénierie des fonctionnalités. Plus précisément, vous allez :

* ingérer des données depuis un répertoire de base avec `ExampleGen`.
* Calculer les statistiques des données d'entraînement avec `StatisticsGen`.
* Déduire un schéma avec `SchemaGen`.
* Détectez les anomalies dans les données d'évaluation avec `ExampleValidator`.
* Prétraiter les données en caractéristiques adaptées à l'apprentissage du modèle avec `Transform`.

Si plusieurs des étapes mentionnées ci-dessus vous semblent familières, c'est parce que les composants TFX qui traitent de la validation et de l'analyse des données (c'est-à-dire `StatisticsGen`, `SchemaGen`, `ExampleValidator`) utilisent [Tensorflow Data Validation (TFDV)](https://www.tensorflow.org/tfx/data_validation/get_started) sous le capot. Vous êtes déjà familiarisé avec cette bibliothèque grâce aux exercices de la semaine 1 et pour cette semaine, vous allez voir comment elle s'intègre dans un pipeline ML.

Les composants que vous utiliserez sont les cases orange mises en évidence dans la figure ci-dessous :

<img src='img/feature_eng_pipeline.png'>



## Setup

### Import packages

Let's begin by importing the required packages and modules. In case you want to replicate this in your local workstation, we used *Tensorflow v2.6* and *TFX v1.3.0*.

In [ ]:
import tensorflow as tf

from tfx import v1 as tfx

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

import os
import pprint
pp = pprint.PrettyPrinter()

### Define paths

Vous allez définir quelques variables globales pour indiquer les chemins dans l'espace de travail local.

In [ ]:
# location of the pipeline metadata store
_pipeline_root = './pipeline/'

# directory of the raw data files
_data_root = './data/census_data'

# path to the raw training data
_data_filepath = os.path.join(_data_root, 'adult.data')

### Prévisualiser le jeu de données

Vous utiliserez à nouveau le [Census Income dataset](https://archive.ics.uci.edu/ml/datasets/Adult) du laboratoire non noté de la semaine 1 afin de pouvoir comparer les résultats lorsque vous utilisez TFDV de façon autonome et lorsque vous l'utilisez sous TFX. Pour rappel, les données peuvent être utilisées pour prédire si un individu gagne plus ou moins de 50k Dollars US par an. Voici à nouveau la description des caractéristiques : 


* **âge** : continu.
* **classe de travail** : Privé, Indépendant-non-inc, Indépendant-inc, Fédéral-gouv, Local-gouv, Etat-gouv, Sans-payé, Jamais-travaillé.
* **fnlwgt** : continu.
* ** **éducation** : Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
* ** **numéro d'éducation** : continu.
* **Statut matrimonial** : Marié(e)-conjoint(e), Divorcé(e), Jamais marié(e), Séparé(e), Veuf(ve), Conjoint(e)-absent(e), Conjoint(e)-AF.
* **Profession** : Support technique, Artisanat-réparation, Autres services, Ventes, Cadres supérieurs, Prof-spécialité, Manutentionnaires-nettoyeurs, Machines-op-inspct, Administration-classe, Agriculture-pêche, Transport-mouvement, Services domestiques privés, Services de protection, Forces armées.
* **relation** : Femme, Enfant à charge, Mari, Pas dans la famille, Autre parent, Célibataire.
* **Race** : Blanc, Asiatique-Pac-Islander, Amérindien-Esquimau, Autre, Noir.
* **sexe** : Femme, Homme.
* **Gain de capital** : continu.
* **perte de capital** : continu.
* **heures-per-week** : continu.
* **Pays d'origine** : Etats-Unis, Cambodge, Angleterre, Puerto-Rico, Canada, Allemagne, Etats-Unis d'Amérique (Guam-USVI-etc), Inde, Japon, Grèce, Sud, Chine, Cuba, Iran, Honduras, Philippines, Italie, Pologne, Jamaïque, Vietnam, Mexique, Portugal, Irlande, France, République dominicaine, Laos, Équateur, Taiwan, Haïti, Colombie, Hongrie, Guatemala, Nicaragua, Écosse, Thaïlande, Yougoslavie, El-Salvador, Trinadad&Tobago, Pérou, Hong, Hollande.

In [ ]:
# preview the first few rows of the CSV file
!head {_data_filepath}

age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
39, State-gov, 77516, Bachelors, 13, Never-married, Adm-clerical, Not-in-family, White, Male, 2174, 0, 40, United-States, <=50K
50, Self-emp-not-inc, 83311, Bachelors, 13, Married-civ-spouse, Exec-managerial, Husband, White, Male, 0, 0, 13, United-States, <=50K
38, Private, 215646, HS-grad, 9, Divorced, Handlers-cleaners, Not-in-family, White, Male, 0, 0, 40, United-States, <=50K
53, Private, 234721, 11th, 7, Married-civ-spouse, Handlers-cleaners, Husband, Black, Male, 0, 0, 40, United-States, <=50K
28, Private, 338409, Bachelors, 13, Married-civ-spouse, Prof-specialty, Wife, Black, Female, 0, 0, 40, Cuba, <=50K
37, Private, 284582, Masters, 14, Married-civ-spouse, Exec-managerial, Wife, White, Female, 0, 0, 40, United-States, <=50K
49, Private, 160187, 9th, 5, Married-spouse-absent, Other-service, Not-in-family, Black, Female, 0, 0,

### Créer le contexte interactif

Lors de la mise en production, vous souhaitez automatiser l'exécution du pipeline en utilisant des orchestrateurs tels que [Apache Beam](https://beam.apache.org/) et [Kubeflow](https://www.kubeflow.org/). Vous ne le ferez pas tout de suite et vous exécuterez le pipeline à partir de ce notebook. Lorsque vous expérimentez dans un environnement de notebook, vous exécuterez *manuellement* les composants du pipeline (c'est-à-dire que vous serez l'orchestrateur). Pour cela, TFX fournit le [Contexte interactif](https://github.com/tensorflow/tfx/blob/master/tfx/orchestration/experimental/interactive/interactive_context.py) afin que vous puissiez parcourir chaque composant et inspecter ses résultats.

Vous allez initialiser le `InteractiveContext` ci-dessous. Cela créera une base de données dans le répertoire `_pipeline_root` que les différents composants utiliseront pour sauvegarder ou obtenir l'état des exécutions des composants. Vous en apprendrez plus à ce sujet dans la semaine 3 lorsque nous discuterons des métadonnées ML. Pour l'instant, vous pouvez le considérer comme le magasin de données qui permet aux différents composants du pipeline de travailler ensemble. 

*Remarque : vous pouvez configurer la base de données à laquelle vous souhaitez vous connecter, mais pour cet exercice, nous utiliserons la base de données par défaut, qui est un fichier sqlite local nouvellement créé*.

In [ ]:
# Initialisez l'InteractiveContext avec un fichier sqlite local.
# Si vous laissez `_pipeline_root` vide, alors le db sera créé dans un répertoire temporaire.

context = InteractiveContext(pipeline_root=_pipeline_root)

## Run TFX components interactively

Avec cela, vous pouvez maintenant exécuter le pipeline de manière interactive. Vous verrez comment le faire en parcourant les différents composants ci-dessous.

### ExampleGen

Vous démarrez le pipeline avec le composant [ExampleGen](https://www.tensorflow.org/tfx/guide/examplegen). Celui-ci va :

* diviser les données en ensembles d'entraînement et d'évaluation (par défaut : 2/3 train, 1/3 eval).
* convertir chaque ligne de données au format `tf.train.Example`. Ce [tampon de protocole](https://developers.google.com/protocol-buffers) est conçu pour les opérations Tensorflow et est utilisé par les composants TFX.
* compresser et sauvegarder la collection de données sous le répertoire `_pipeline_root` pour que les autres composants puissent y accéder. Ces exemples sont stockés au format `TFRecord`. Cela optimise les opérations de lecture et d'écriture dans Tensorflow, surtout si vous avez une grande collection de données.

Son constructeur prend le chemin vers votre source/répertoire de données. Dans notre cas, c'est le chemin `_data_root`. Le composant supporte plusieurs sources de données telles que CSV, tf.Record, et BigQuery. Puisque nos données sont un fichier CSV, nous utiliserons [CsvExampleGen](https://www.tensorflow.org/tfx/api_docs/python/tfx/components/CsvExampleGen) pour ingérer les données.

Exécutez la cellule ci-dessous pour instancier `CsvExampleGen`.

In [ ]:
# Instantiate ExampleGen with the input CSV dataset
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)

Vous pouvez exécuter le composant en appelant la méthode `run()` du `InteractiveContext`.

In [ ]:
# Execute the component
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 15
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 15
        type_id: 14
        uri: "./pipeline/CsvExampleGen/examples/15"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:3974460,xor_checksum:1635233824,sum_checksum:1635233824"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

Vous remarquerez qu'une cellule de sortie montrant les résultats de l'exécution est automatiquement affichée. Ces métadonnées sont enregistrées dans la base de données créée précédemment. Cela vous permet de garder une trace des exécutions de votre projet. Par exemple, si vous l'exécutez à nouveau, vous remarquerez que le `.execution_id` s'incrémente.

Les résultats des composants sont appelés *artifacts* et vous pouvez voir un exemple en naviguant dans `.component.outputs > ['examples'] > Channel > ._artifacts > [0]` ci-dessus. Il montre des informations telles que l'endroit où les données converties sont stockées (`.uri`) et les splits générés (`.split_names`).

Vous pouvez également examiner les artefacts de sortie de manière programmatique avec le code ci-dessous.

In [ ]:
# get the artifact object
artifact = example_gen.outputs['examples'].get()[0]

# print split names and uri
print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')

split names: ["train", "eval"]
artifact uri: ./pipeline/CsvExampleGen/examples/15


Si vous vous posez la question, le `number` dans `./pipeline/CsvExampleGen/examples/{number}` est l'id d'exécution associé à ce jeu de données. Si vous redémarrez le noyau de cet espace de travail et ré-exécutez jusqu'à cette cellule, vous remarquerez qu'un nouveau dossier avec un nom d'id différent a été créé. Cela montre que TFX conserve des versions de vos données afin que vous puissiez revenir en arrière si vous souhaitez étudier une exécution particulière.

Comme mentionné, les données ingérées sont stockées dans le répertoire indiqué dans le champ `uri`. Elles sont également compressées en utilisant `gzip` et vous pouvez le vérifier en exécutant la cellule ci-dessous.

In [ ]:
# Get the URI of the output artifact representing the training examples
train_uri = os.path.join(artifact.uri, 'Split-train')

# See the contents of the `train` folder
!ls {train_uri}

data_tfrecord-00000-of-00001.gz


Dans un environnement de notebook, il peut être utile d'examiner quelques exemples de données, surtout si vous êtes encore en train d'expérimenter. Comme la collection de données est enregistrée au format [TFRecord](https://www.tensorflow.org/tutorials/load_data/tfrecord), vous devrez utiliser des méthodes qui fonctionnent avec ce type de données. Vous devrez décompresser les exemples individuels du fichier `TFRecord` et le formater pour l'impression. Faisons cela dans les cellules suivantes :

In [ ]:
# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")


In [ ]:
# Define a helper function to get individual examples
def get_records(dataset, num_records):
    
    '''
    Extrait les enregistrements de l'ensemble de données donné.
    Args :
        dataset (TFRecordDataset) : jeu de données enregistré par ExampleGen
        num_records (int) : nombre d'enregistrements à prévisualiser
    '''
    
    # initialize an empty list
    records = []
    
    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))
        
        # append to the records list
        records.append(example_dict)
        
    return records

In [ ]:
# Get 3 records from the dataset
sample_records = get_records(dataset, 3)

# Print the output
pp.pprint(sample_records)

[{'features': {'feature': {'age': {'int64List': {'value': ['39']}},
                           'capital-gain': {'int64List': {'value': ['2174']}},
                           'capital-loss': {'int64List': {'value': ['0']}},
                           'education': {'bytesList': {'value': ['IEJhY2hlbG9ycw==']}},
                           'education-num': {'int64List': {'value': ['13']}},
                           'fnlwgt': {'int64List': {'value': ['77516']}},
                           'hours-per-week': {'int64List': {'value': ['40']}},
                           'label': {'bytesList': {'value': ['IDw9NTBL']}},
                           'marital-status': {'bytesList': {'value': ['IE5ldmVyLW1hcnJpZWQ=']}},
                           'native-country': {'bytesList': {'value': ['IFVuaXRlZC1TdGF0ZXM=']}},
                           'occupation': {'bytesList': {'value': ['IEFkbS1jbGVyaWNhbA==']}},
                           'race': {'bytesList': {'value': ['IFdoaXRl']}},
                    

Maintenant que `ExampleGen` a fini d'ingérer les données, l'étape suivante est l'analyse des données.

### StatisticsGen

Le composant [StatisticsGen](https://www.tensorflow.org/tfx/guide/statsgen) calcule des statistiques sur votre ensemble de données pour l'analyse des données, ainsi que pour une utilisation dans les composants en aval (c'est-à-dire les étapes suivantes du pipeline). Comme mentionné précédemment, ce composant utilise TFDV sous le capot, donc sa sortie vous sera familière.

`StatisticsGen` prend en entrée le jeu de données que nous venons d'ingérer en utilisant `CsvExampleGen`.

In [ ]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])

# Execute the component
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 16
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 16
        type_id: 16
        uri: "./pipeline/StatisticsGen/statistics/16"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

Vous pouvez afficher les statistiques avec la méthode `show()`.


In [ ]:
# Show the output statistics
context.show(statistics_gen.outputs['statistics'])

### SchemaGen

Le composant [SchemaGen](https://www.tensorflow.org/tfx/guide/schemagen) utilise également TFDV pour générer un schéma basé sur les statistiques de vos données. Comme vous l'avez appris précédemment, un schéma définit les limites, les types et les propriétés attendus des caractéristiques de votre jeu de données.

`SchemaGen` prendra comme entrée les statistiques que nous avons générées avec `StatisticsGen`, en regardant le split d'entraînement par défaut.

In [ ]:
# Instantiate SchemaGen with the StatisticsGen ingested dataset
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )

# Run the component
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 17
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 17
        type_id: 18
        uri: "./pipeline/SchemaGen/schema/17"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

You can then visualize the generated schema as a table.

In [ ]:
# Visualize the schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'education',STRING,required,,'education'
'label',STRING,required,,'label'
'marital-status',STRING,required,,'marital-status'
'native-country',STRING,required,,'native-country'
'occupation',STRING,required,,'occupation'
'race',STRING,required,,'race'
'relationship',STRING,required,,'relationship'
'sex',STRING,required,,'sex'
'workclass',STRING,required,,'workclass'


,Values
Domain,
'education',"' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate', ' HS-grad', ' Masters', ' Preschool', ' Prof-school', ' Some-college'"
'label',"' <=50K', ' >50K'"
'marital-status',"' Divorced', ' Married-AF-spouse', ' Married-civ-spouse', ' Married-spouse-absent', ' Never-married', ' Separated', ' Widowed'"
'native-country',"' ?', ' Cambodia', ' Canada', ' China', ' Columbia', ' Cuba', ' Dominican-Republic', ' Ecuador', ' El-Salvador', ' England', ' France', ' Germany', ' Greece', ' Guatemala', ' Haiti', ' Honduras', ' Hong', ' Hungary', ' India', ' Iran', ' Ireland', ' Italy', ' Jamaica', ' Japan', ' Laos', ' Mexico', ' Nicaragua', ' Outlying-US(Guam-USVI-etc)', ' Peru', ' Philippines', ' Poland', ' Portugal', ' Puerto-Rico', ' Scotland', ' South', ' Taiwan', ' Thailand', ' Trinadad&Tobago', ' United-States', ' Vietnam', ' Yugoslavia', ' Holand-Netherlands'"
'occupation',"' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair', ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners', ' Machine-op-inspct', ' Other-service', ' Priv-house-serv', ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support', ' Transport-moving'"
'race',"' Amer-Indian-Eskimo', ' Asian-Pac-Islander', ' Black', ' Other', ' White'"
'relationship',"' Husband', ' Not-in-family', ' Other-relative', ' Own-child', ' Unmarried', ' Wife'"
'sex',"' Female', ' Male'"
'workclass',"' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'"


Passons maintenant à l'étape suivante du pipeline et voyons s'il y a des anomalies dans les données.

### ExampleValidator

Le composant [ExampleValidator](https://www.tensorflow.org/tfx/guide/exampleval) détecte les anomalies dans vos données sur la base du schéma généré à l'étape précédente. Comme les deux composants précédents, il utilise également TFDV sous le capot. 

`ExampleValidator` prend en entrée les statistiques de `StatisticsGen` et le schéma de `SchemaGen`. Par défaut, il compare les statistiques de l'évaluation au schéma de l'entraînement.

In [ ]:
# Instantiate ExampleValidator with the StatisticsGen and SchemaGen ingested data
example_validator = tfx.components.ExampleValidator(
    
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

# Run the component.
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 18
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 18
        type_id: 20
        uri: "./pipeline/ExampleValidator/anomalies/18"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

Comme pour le composant précédent, vous pouvez également visualiser les anomalies sous forme de tableau.

In [ ]:
# Visualize the results
context.show(example_validator.outputs['anomalies'])

With no anomalies detected, you can proceed to the next step in the pipeline.

### Transform
Le composant [Transform](https://www.tensorflow.org/tfx/guide/transform) effectue l'ingénierie des caractéristiques pour les ensembles de données de formation et de service. Il utilise la bibliothèque [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started) introduite dans le premier laboratoire non noté de cette semaine.

`Transform` prendra comme entrée les données de `ExampleGen`, le schéma de `SchemaGen`, ainsi qu'un module contenant la fonction de prétraitement.

Dans cette section, vous allez travailler sur un exemple de code Transform défini par l'utilisateur. Le pipeline doit charger ce code en tant que module, vous devez donc utiliser la commande magique `%% writefile` pour enregistrer le fichier sur le disque. Définissons d'abord quelques constantes qui regroupent les attributs des données en fonction des transformations que nous effectuerons plus tard. Ce fichier sera également sauvegardé localement.

In [ ]:
# Set the constants module filename
_census_constants_module_file = 'census_constants.py'

In [ ]:
%%writefile {_census_constants_module_file}

# Features with string data types that will be converted to indices
CATEGORICAL_FEATURE_KEYS = [
    'education', 'marital-status', 'occupation', 'race', 'relationship', 'workclass', 'sex', 'native-country'
]

# Numerical features that are marked as continuous
NUMERIC_FEATURE_KEYS = ['fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

# Feature that can be grouped into buckets
BUCKET_FEATURE_KEYS = ['age']

# Number of buckets used by tf.transform for encoding each bucket feature.
FEATURE_BUCKET_COUNT = {'age': 4}

# Feature that the model will predict
LABEL_KEY = 'label'

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Writing census_constants.py


Ensuite, vous allez travailler sur le module qui contient `preprocessing_fn()`. Comme vous l'avez vu dans le laboratoire précédent, cette fonction définit comment vous allez transformer les données brutes en caractéristiques sur lesquelles votre modèle peut s'entraîner (c'est-à-dire l'étape suivante dans le pipeline). Vous utiliserez les [fonctions du module tft](https://www.tensorflow.org/tfx/transform/api_docs/python/tft) pour effectuer ces transformations. 

In [ ]:
# Set the transform module filename
_census_transform_module_file = 'census_transform.py'

In [ ]:
%%writefile {_census_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import census_constants

# Unpack the contents of the constants module
_NUMERIC_FEATURE_KEYS = census_constants.NUMERIC_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = census_constants.CATEGORICAL_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = census_constants.BUCKET_FEATURE_KEYS
_FEATURE_BUCKET_COUNT = census_constants.FEATURE_BUCKET_COUNT
_LABEL_KEY = census_constants.LABEL_KEY
_transformed_name = census_constants.transformed_name


# Define the transformations
def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    outputs = {}

    # Scale these features to the range [0,1]
    for key in _NUMERIC_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.scale_to_0_1(
            inputs[key])
    
    # Bucketize these features
    for key in _BUCKET_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.bucketize(
            inputs[key], _FEATURE_BUCKET_COUNT[key])

    # Convert strings to indices in a vocabulary
    for key in _CATEGORICAL_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(inputs[key])

    # Convert the label strings to an index
    outputs[_transformed_name(_LABEL_KEY)] = tft.compute_and_apply_vocabulary(inputs[_LABEL_KEY])

    return outputs

Writing census_transform.py


Vous pouvez maintenant passer les données de formation, le schéma et le module de transformation au composant `Transform`. Vous pouvez ignorer les messages d'avertissement générés par Apache Beam concernant les indications de type.

In [ ]:
# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')

# Instantiate the Transform component
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_census_transform_module_file))

# Run the component
context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 20
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 19
        type_id: 22
        uri: "./pipeline/Transform/transform_graph/19"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1677949978722
        last_update_time_since_epoch: 1677950012109
        , artifact_type: id: 22
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 20
        type_id: 14
        uri: "./pipeline/Transform/transformed_examples/19"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1677949978737
        last_update_time_since_epoch: 1677950012110
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 21
        type_id: 23
        uri: "./pipeline/Transform/updated_analyzer_cache/19"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1677949978737
        last_update_time_since_epoch: 1677950012110
        , artifact_type: id: 23
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 22
        type_id: 18
        uri: "./pipeline/Transform/pre_transform_schema/19"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
       

Examinons les artefacts de sortie de `Transform` (c'est-à-dire `.component.outputs` de la cellule de sortie ci-dessus). Ce composant produit plusieurs sorties :

* `transform_graph` est le graphe qui peut effectuer les opérations de prétraitement. Ce graphe sera inclus pendant l'entraînement et servira à assurer des transformations cohérentes des données entrantes.
* `transformed_examples` pointe vers les données d'entraînement et d'évaluation prétraitées.
* `updated_analyzer_cache` sont les calculs stockés des exécutions précédentes.

Take a peek at the `transform_graph` artifact.  It points to a directory containing three subdirectories.

In [ ]:
# Get the uri of the transform graph
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri

# List the subdirectories under the uri
os.listdir(transform_graph_uri)

['metadata', 'transformed_metadata', 'transform_fn']

* Le sous-répertoire `metadata` contient le schéma des données originales.
* Le sous-répertoire `transformed_metadata` contient le schéma des données prétraitées. 
* Le sous-répertoire `transform_fn` contient le graphique de prétraitement réel. 

Vous pouvez également jeter un coup d'oeil aux trois premiers exemples transformés en utilisant la fonction d'aide définie précédemment.

In [ ]:
# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [ ]:
# Get 3 records from the dataset
sample_records_xf = get_records(transformed_dataset, 3)

# Print the output
pp.pprint(sample_records_xf)

[{'features': {'feature': {'age_xf': {'int64List': {'value': ['2']}},
                           'capital-gain_xf': {'floatList': {'value': [0.021740217]}},
                           'capital-loss_xf': {'floatList': {'value': [0.0]}},
                           'education-num_xf': {'floatList': {'value': [0.8]}},
                           'education_xf': {'int64List': {'value': ['2']}},
                           'fnlwgt_xf': {'floatList': {'value': [0.044301897]}},
                           'hours-per-week_xf': {'floatList': {'value': [0.39795917]}},
                           'label_xf': {'int64List': {'value': ['0']}},
                           'marital-status_xf': {'int64List': {'value': ['1']}},
                           'native-country_xf': {'int64List': {'value': ['0']}},
                           'occupation_xf': {'int64List': {'value': ['3']}},
                           'race_xf': {'int64List': {'value': ['0']}},
                           'relationship_xf': {'int64List